In [ ]:
https://sparkbyexamples.com/pyspark/pyspark-rdd-transformations/
https://sparkbyexamples.com/pyspark-tutorial/
https://spark.apache.org/docs/latest/rdd-programming-guide.html

Recap
======
PySpark 
- follows Master-Slave design
- key components - python+java+scala 
- pyspark - use lib - Py4j
- Py4j - bridge between Python and Core spark components - Scala - run JVM


from pyspark.sql import SparkSession
SparkSession.builder.appName('userdefinedAppName').getOnCreate() //sparkContext
sparkContext.createDataFrame()

DataFrame
RDD 
Transformation - filter select map ....
action - execution -> show() count()

CPU-2-core
4GB RAM
    |
    open("data.csv","r")
           |
          5GB size

    pandas - pd.read_csv('data.csv')
                          |->5GB
    data.csv
      |<---- live data -   registrationForm
                            --<---
      <--- append to file
================================================================
app       event_records/
   |---->   10_.log       <--------------- load this data
   |---->   10.05_log
               ...
================================================================

1. Initialize spark session object
2. read 
3. transformation 
4. action
5. stop session

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()

data = [("pA",101),("pB",102),("pC",103)]
columns = ["pName","pID"]

df = spark.createDataFrame(data,columns)
#f.show()

In [6]:
spark

In [4]:
#spark.stop()

In [ ]:
df1 = spark.read.csv('prod.csv',header=True) = from local node

df2 = spark.read.csv('hdfs://namenode:9000/prod.csv',header=True) = from HDFS

df3 = spark.read.csv("s3a://bucket/data.csv") = from Cloud object


In [ ]:
>>>
>>> from pyspark.sql import SparkSession
>>>
>>> spark_session_obj = SparkSession.builder.appName("rdd-demo").getOrCreate()
25/11/05 06:22:12 WARN Utils: Your hostname, paka resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/05 06:22:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/05 06:22:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
>>>
>>> sc = spark_session_obj.sparkContext
>>>
>>> L = [1,2,3,4,5]
>>> sc.parallelize(L)
ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289
>>>
>>> rdd_obj = sc.parallelize(L)
>>> rdd_obj.map(lambda a:a*5)
PythonRDD[2] at RDD at PythonRDD.scala:53
>>>
>>> rdd1 = rdd_obj.map(lambda a:a*5)
>>>
>>> rdd2 = rdd_obj.filter(lambda a: a>3)
>>>
>>> rdd1.collect()
[5, 10, 15, 20, 25]
>>> rdd2.collect()
[4, 5]
>>>
>>> rdd3 = rdd1.filter(lambda a:a>10)
>>> rdd3.collect()
[15, 20, 25]
>>>
>>> file_rdd = sc.textFile("prod.csv")
>>>


In [ ]:
# Task
1. initialize sparksession object
2. initialize spark context from session object
3. using spark_context object read a any text file(.log .csv .txt..) # use sc.textFile('filename')
4. using collect() - display file contents
----
5. use flatMap() - split each line into multiple items(words)
   display splitted results 
----
6. use map() - convert each word into tuple
   display map results
---
7. use reduceByKey() - group all values by their key(each word)
   then it reduces(aggregates) values for each key using reduceByKey() function.
--------------

For Example - input data file is data.log
(pyspark-env) student@paka:~$ cat >data.log
project details
list of loaded data model
linear data set value
list of product details
data set value
(pyspark-env) student@paka:~$

=========================================================================

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test1").getOrCreate()
sc = spark.sparkContext
rdd = sc.textFile('data.log')
print(rdd.collect())
print('') # empty line
#####################################
rdd2 = rdd.flatMap(lambda a:a.split(" "))
print(rdd2.collect())
print('') # empty line
#####################################
rdd3 = rdd2.map(lambda a:(a,1))
print(rdd3.collect())
print('') # empty line
####################################
print(rdd3.reduceByKey(lambda a,b:a+b).collect())
################################################


In [ ]:
[10,20,30,40]

[10,20]  - partition1
[30,40]  - partition2
-----------------------//RDD - split the data into smaller chunk(partition)

number of partitions = number of cpu core


[10,20]  - partition1 -- spark sends each partition to different executor (jvm)
[30,40]  - partition2 -- spark sends each partition to different executor (jvm) 
|
transformation ->action
---------------------------------

userInput - inputlist/collection/dataset 
|
DriverProgram
|
SparkContext
|
+---------------------+
| Partition1:[10,20]  ----------> | executor - jvm1 | + 100 =>[110,120]
+---------------------+
| Partition2:[30,40]  ----------> | executor - jvm2 | + 100 =>[130,140]
+---------------------+
    |
    Result -- collected back ==> [110,120,130,140]

In [ ]:
1. Batch Processing
=====================
Process - finite data(complete dataset) - stored in DB/file - fetch the complete data do 
                                                              process at once.

df = pd.read_csv('emp.csv')
df.shape ->(10,5) Ex: executed on time 10:05 - df.shape
df.shape ->(10,5) Ex: executed on time 10:10 - df.shape
# vi emp.csv
# added two more lines 
# :wq 
# ------------- time on 10:15 
df.shape ->(10,5) Ex: executed on time 10:20 - df.shape 

Vs
2. Stream Processing
=====================
Process - infinite data (or) real-time data


    +----------------------------------+
    | Streaming Source  <== Socket,File,Kafka etc.,
    |                       ---(input source)---
    +-----------|-----------------------+
    | SparkStructured Streaming         |
    |    (Transformation)               |
    +-----------|-----------------------+
    |   Streming Sink ---->Console,HDFS,Delta Lake,Kafa etc.,
    |                       (Output sink)
    +-----------------------------------+

    Stream output modes
    |->append mode  --- adds only new records/row to result table - not using aggregate method
    |->update mode --- increment value/recent value  - aggregate
    |->complete mode -- full value - aggregate


1st - spark session object
2nd - spark session object interface to inputmode  ->df
3rd - df.do_Transformation ->result
4th - result ->Sink_to_output


DataStream                        UnboundedTable
--------------                    ------------------
  ----------------->                  ...
                                   ----------------

 ------------->|Sparksteam | -->[][][][] ---->|Spark core |-->outputSinks
                              inputbatches 

In [ ]:
data = [
    (101,"pA",1000,"USA"),
    (102,"pB",2000,"Canada"),
    (103,"pC",3000,"India"),
    (104,"pD",4000,"USA"),
    (105,"pE",3400,"India")
]

In [ ]:
>>> from pyspark.sql import SparkSession
>>> from pyspark.sql.functions import col
>>> data = [("pA",101,1000),("pB",102,2000),("pC",103,3000)]
>>>
>>> columns = ["pname","pid","pcost"]
>>>
>>> df = spark_obj.createDataFrame(data,columns)
>>> df.show()
+-----+---+-----+
|pname|pid|pcost|
+-----+---+-----+
|   pA|101| 1000|
|   pB|102| 2000|
|   pC|103| 3000|
+-----+---+-----+
>>> new_df = df.withColumn("Tax",col("pcost") * 0.12)
>>>
>>> new_df.show()
+-----+---+-----+-----+
|pname|pid|pcost|  Tax|
+-----+---+-----+-----+
|   pA|101| 1000|120.0|
|   pB|102| 2000|240.0|
|   pC|103| 3000|360.0|
+-----+---+-----+-----+

>>> # df['tax']=df['pcost']*0.12 # in pandas
>>> # ----------------------------------------


In [ ]:
1. Create a dataframe
2. use the above dataframe ->select only the product and price columns - cost above 3500
    Hint: select() filter() where()
3. Add new column -> withColumn() - transformation
                     df.withColumn("<ColumnName>",ColValue)
|
4. Aggregation and Grouping 
    -> df.groupBy("country").sum() / avg()
5. display

-----------------------------------------------------

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark_obj = SparkSession.builder.appName("Activity-1").getOrCreate()

data = [
    (101,"pA",1000,"USA"),
    (102,"pB",2000,"Canada"),
    (103,"pC",3000,"India"),
    (104,"pD",4000,"USA"),
    (105,"pE",3400,"India")
]
columns = ["productID","product","cost","country"]

prod_df = spark_obj.createDataFrame(data,columns)
print(prod_df.show())

r1 = prod_df.filter((col("country") == "India") & (col("cost") >2560))
print(r1.show())

r2 = r1.select("product","cost")
print(r2.show())

# To add new column 
final_df = prod_df.withColumn("Tax",col("cost") * 0.18)
final_df.show()

# Aggregation 
r3 = prod_df.groupBy("country").sum("cost")
r3.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,desc

spark_obj = SparkSession.builder.appName("Activity-1").getOrCreate()

data = [
    (101,"pA",1000,"USA"),
    (102,"pB",2000,"Canada"),
    (103,"pC",3000,"India"),
    (104,"pD",4000,"USA"),
    (105,"pE",3400,"India")
]
columns = ["productID","product","cost","country"]

prod_df = spark_obj.createDataFrame(data,columns)
prod_df.show()

r1 = prod_df.filter((col("country") == "India") & (col("cost") >2560))
r1.show()

r2 = r1.select("product","cost")
r2.show()
# To add new column
final_df = prod_df.withColumn("Tax",col("cost") * 0.18)
final_df.show()


# Aggregation
r3 = prod_df.groupBy("country").sum("cost")
r3.show()

In [ ]:

    +----------------------------------+
    | Streaming Source  <== Socket,File,Kafka etc.,  - readStream.format().<config>.load()
    |                       ---(input source)---
    +-----------|-----------------------+
    | SparkStructured Streaming         |
    |    (Transformation)               |
    +-----------|-----------------------+
    |   Streming Sink ---->Console,HDFS,Delta Lake,Kafa etc., - writeStream.format().start()
    |                       (Output sink)
    +-----------------------------------+

In [ ]:
## Stream 
from pyspark.sql import SparkSession
spark_obj = SparkSession.builder.appName("socketdemo").master("local[*]").getOrCreate()
stream_df = spark_obj.readStream.format("socket").option("host","localhost").option("port","1100").load()

print(stream_df.isStreaming) # ->bool(True,False)
print(stream_df.printSchema())


In [ ]:
input stream source is file => input_directory/ 
                                              |->newfile1 |->newfile2 ...
define the schema  =>

defined data type -> from pyspark.sql.types import StructType
                                                      |->StringType,IntegerType...



In [ ]:
readStream.format("socket")  .. ->df
df.writeStream.outputMode("append").format("csv")......

In [ ]:
WebApp -- producer
 |
Kafka
|
Analytics service - consumer 


Kafka is running -(on in docker)
|
kafka => kafa-topic.sh --create --topic transactions --bootstarp-server localhost:9092 {Enter}
kafa-console-producer.sh --topic transaction --bootstrap-server localhost:9092 {enter}
  data - json 
      {"pname":"pA","pid":1000,...}
--------------------------------------
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

spark = SparkSession.builder.appName("filestream").getOrCreate()

schema_obj = StructType([StructField("pname",StringType(),True),StructField("pid",IntegerType(),True)])

kafaka_stream = spark.readStream.format("kafka").option("kafaka.bootstarp.servers","localhost:9092")\
.option(..).load()

..
kafaka_stream.writeStream.outputMode("complete").format("console").start().awaitTermination()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()

data = [("pA",101),("pB",102),("pC",103)]
columns = ["pName","pID"]

df = spark.createDataFrame(data,columns)
df.show()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()

data = [("pA",101),("pB",102),("pC",103)]
columns = ["pName","pID"]

df = spark.createDataFrame(data,columns)
df.show()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()

data = [("pA",101),("pB",102),("pC",103)]
columns = ["pName","pID"]

df1 = spark.createDataFrame(data,columns)

df2 = spark.read.csv('prod.csv',header=True)
df1.printSchema()
df2.printSchema()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()


df = spark.read.csv('prod.csv',header=True)
df.select("productName").show()
df.filter(df.productCost >1250).show()
df.groupBy("productID").count().show()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').getOrCreate()


df = spark.read.csv('prod.csv',header=True)
df.select("productName").show()
df.filter(df.productCost >1250).show()
df.groupBy("productID").count().show()

df.createOrReplaceTempView("prod")
result = spark.sql("select *from prod where productCost >3000")
result.show()

In [ ]:
## rdd examples

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo2").getOrCreate()
sc = spark.sparkContext
rdd = sc.parallelize([10,20,30,40,50])
print("partition count:",rdd.getNumPartitions())
rdd = sc.parallelize([10,20,30,40,50],4)
print("partition count:",rdd.getNumPartitions())

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo2").getOrCreate()
sc = spark.sparkContext

# create an RDD with 2 partitions
rdd = sc.parallelize([10,20,30,40,50,60],2)

# To get partitions count
print(rdd.getNumPartitions())

rdd1 = rdd.map(lambda a:a+100)
########################### immutable  - not changable - initialize to another rdd

print(rdd1.collect()) # collect and display results

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo2").getOrCreate()
sc = spark.sparkContext

# create an RDD with 2 partitions
rdd = sc.parallelize([10,20,30,40,50,60],2)

mapped_obj = rdd.map(lambda a:[a,a+100])
print(mapped_obj.collect())
print('') # empty line

rdd = sc.parallelize([10,20,30,40,50,60],2)
flat_mapped_obj = rdd.flatMap(lambda a:[a,a+100])
print(flat_mapped_obj.collect())

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo2").getOrCreate()
sc = spark.sparkContext

rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([4,5,6])
rdd_result = rdd1.union(rdd2)
print(rdd_result.collect())
print("")
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([3,5,6])
rdd_result = rdd1.union(rdd2)
print(rdd_result.collect())

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo2").getOrCreate()
sc = spark.sparkContext
rdd = sc.parallelize([("row1",10),("row2",20),("row3",30),("row1",40),("row2",50)])
print(rdd.reduceByKey(lambda a,b:a+b).collect())

In [ ]:
# PySpark with Delta Lake
# --------------------------

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DeltaLakedemo1").config("spark.sql.extension","io.delta.sql.DeltaSparkSessionException").config("spark.sql.catalog.spark_catlog","org.apache.spark.sql.delta.catalog.DeltaCatalog").getOrCreate()

data = [("pA",101),("pB",102),("pC",103)]
df = spark.createDataFrame(data,["pname","pid"])

df.write.format("delta").save("/tmp/delta-table")

# Read-Delta table
#spark.read.format("delta").load("./delta-table").show()

In [ ]:
(pyspark-env) student@paka:~$ cat rdd_activity1.py
'''
# Task
1. initialize sparksession object
2. initialize spark context from session object
3. using spark_context object read a any text file(.log .csv .txt..) # use sc.textFile('filename')
4. using collect() - display file contents
----
5. use flatMap() - split each line into multiple items(words)
   display splitted results
----
6. use map() - convert each word into tuple
   display map results
---
7. use reduceByKey() - group all values by their key(each word)
   then it reduces(aggregates) values for each key using reduceByKey() function.
--------------

For Example - input data file is data.log
(pyspark-env) student@paka:~$ cat >data.log
project details
list of loaded data model
linear data set value
list of product details
data set value
(pyspark-env) student@paka:~$

'''

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("test1").getOrCreate()
sc = spark.sparkContext
rdd = sc.textFile('data.log')
print(rdd.collect())
print('') # empty line
#####################################
rdd2 = rdd.flatMap(lambda a:a.split(" "))
print(rdd2.collect())
print('') # empty line
#####################################
rdd3 = rdd2.map(lambda a:(a,1))
print(rdd3.collect())
print('') # empty line
####################################
print(rdd3.reduceByKey(lambda a,b:a+b).collect())
################################################

In [ ]:
### streaming examples

In [ ]:
# socket stream
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').master('local[*]').getOrCreate()

df = spark.readStream.format("socket").option("host","localhost").option("port",1120).load()
print(df.isStreaming) # ->bool(True/False)
print(df.printSchema())
write_query = df.writeStream.format("console").start()
write_query.awaitTermination() # keep on running

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('demo1').master('local[*]').getOrCreate()

df = spark.readStream.format("socket").option("host","localhost").option("port",1120).load()
print(df.isStreaming) # ->bool(True/False)
print(df.printSchema())
###
df_result = df.select(explode(split("value"," ")).alias("word"))

word_count = df_result.groupBy("word").count()
###
#
# write_query = word_count.writeStream.format("console").start()
# Error - default outputMode is append

write_query = word_count.writeStream.outputMode("update").format("console").start()
write_query.awaitTermination() # keep on running

In [ ]:
# filestream examples
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('demo1').master('local[*]').getOrCreate()

df = spark.readStream.format("socket").option("host","localhost").option("port",1120).load()
print(df.isStreaming) # ->bool(True/False)
print(df.printSchema())
###
df_result = df.select(explode(split("value"," ")).alias("word"))

word_count = df_result.groupBy("word").count()
###
#
# write_query = word_count.writeStream.format("console").start()
# Error - default outputMode is append

write_query = word_count.writeStream.outputMode("complete").format("console").start()
write_query.awaitTermination() # keep on running

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

spark = SparkSession.builder.appName("filestream").getOrCreate()

schema_obj = StructType([StructField("pname",StringType(),True),StructField("pid",IntegerType(),True)])

file_stream = spark.readStream.option("header","true").schema(schema_obj).csv("input_dir/")

r = file_stream.filter(file_stream.pid >100).groupBy("pname").count()

r.writeStream.outputMode("update").format("console").start().awaitTermination()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

spark = SparkSession.builder.appName("filestream").getOrCreate()

schema_obj = StructType([StructField("pname",StringType(),True),StructField("pid",IntegerType(),True)])

file_stream = spark.readStream.option("header","true").schema(schema_obj).csv("input_dir/")

#r = file_stream.filter(file_stream.pid >100).groupBy("pname").count()

file_stream.writeStream.outputMode("append").format("console").start().awaitTermination()
#r.writeStream.outputMode("append").format("console").start().awaitTermination()
#r.writeStream.format("console").start().awaitTermination()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

spark = SparkSession.builder.appName("jsonstream").getOrCreate()

schema_obj = StructType([StructField("pname",StringType(),True),StructField("pid",IntegerType(),True)])

json_stream = spark.readStream.schema(schema_obj).json("json_dir/")

#r = file_stream.filter(file_stream.pid >100).groupBy("pname").count()

json_stream.writeStream.outputMode("append").format("console").start().awaitTermination()
#r.writeStream.outputMode("append").format("console").start().awaitTermination()
#r.writeStream.format("console").start().awaitTermination()


In [ ]:
## from socket stream -->filestream - examples

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('demo1').master('local[*]').getOrCreate()

df = spark.readStream.format("socket").option("host","localhost").option("port",1120).load()

df.writeStream.outputMode("append").format("csv").option("path","output_dir/").option("checkpointLocation","checkpoints/csv_stream_chpt").option("header","true").start().awaitTermination()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('demo1').master('local[*]').getOrCreate()

df = spark.readStream.format("socket").option("host","localhost").option("port",1120).load()

df_result = df.select(explode(split("value"," ")).alias("word"))

wc = df_result.groupBy("word").count()
wc.writeStream.outputMode("update").format("csv").option("path","output_dir/").option("checkpointLocation","checkpoints/csv_stream_chpt").option("header","true").start().awaitTermination()

In [ ]:
####################

In [ ]:
Airflow 
========
 |->open-source - schedule and monitor workflows (data pipelines)
                     |->daily,hourly,custom schedule
 |-> On linux -> cronjob - - jobschedule 

 |-> airflow ->programming way job schedule and monitor workflow - webUI
                  |->python 
  code sequence ->their dependencies ->schedules -> webUI

 |->Directed Acyclic Graph (DAG)
    ============================
       [task1] -->[task2] -->[task3] --->[task4]  # there is no reverse ; no looping

    DAG - descibes how to run the workflow

 |-> task - code(or)unit 

 |-> operators - pre-defined templates for tasks 
         |-> BashOperator - executes a bash command/script file
         |-> PythonOperator - (@task decoratror) - function
         |-> SqlOperator - Execute SQL command 

 |-> executor/workers - instance 

 |-> webUI 

 |->services
        |->scheduler 
        |->webserver 

---------------------------------------------------
    airflow/
            |->logs/
            |->data/
            |->dags/ <==
                 |<-- dagscript.py

------------------------------------------------------------------------
Create airflow login 
|
airflow users create --username <admin> --firstname Admin --lastname user --role admin
             --email admin@example.com --password <password>{enter}
|
             Login:
             password:
------------------------------------------------------------------------
# Start webserver => airflow webserver --port 8080 {Enter} 

# Start scheduler -> airflow scheduler 

In [ ]:
student@paka:~$ python3 -m venv lab2
student@paka:~$ source lab2/bin/activate
(lab2) student@paka:~$
(lab2) student@paka:~$ export AIRFLOW_VERSION=2.10.1
(lab2) student@paka:~$ export PYTHON_VERSION="$(python --version|cut -d" " -f2|cut -d"." -f 1,2)"
(lab2) student@paka:~$ export CONSTRAINT_URL="https://raw.githubusercontent.com/apache/airflow/constraints-${AIRFLOW_VERSION}/constraints-${PYTHON_VERSION}.txt"
(lab2) student@paka:~$
(lab2) student@paka:~$ pip install "apache-airflow == ${AIRFLOW_VERSION}" --constraint "${CONSTRAINT_URL}"
Collecting apache-airflow==2.10.1
...
(lab2) student@paka:~$ airflow version
2.10.1
(lab2) student@paka:~$
(lab2) student@paka:~$ ls -t
airflow <=== 


(lab2) student@paka:~$ ls airflow
logs
(lab2) student@paka:~$ airflow db init
/home/student/lab2/lib/python3.12/site-packages/airflow/utils/providers_configuration_loader.py:55 DeprecationWarning: `db init` is deprecated.  Use `db migrate` instead to migrate the db and/or airflow connections create-default-connections to create the default connections
DB: sqlite:////home/student/airflow/airflow.db
[2025-11-05T12:17:08.492+0000] {migration.py:215} INFO - Context impl SQLiteImpl.
[2025-11-05T12:17:08.494+0000] {migration.py:218} INFO - Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running stamp_revision  -> 22ed7efa9da2
WARNI [airflow.models.crypto] empty cryptography key - values will not be stored encrypted.
Initialization done
(lab2) student@paka:~$ ls airflow
airflow.cfg  airflow.db  logs
--------------------------------//this is no dags directory 

(lab2) student@paka:~$ ls airflow
airflow.cfg  airflow.db  logs
(lab2) student@paka:~$ mkdir -p ~/airflow/dags # create new dags directory
(lab2) student@paka:~$ ls airflow
airflow.cfg  airflow.db  dags  logs
(lab2) student@paka:~$   ====

airflow users create --username admin --firstname stduent --lastname user 
--role Admin --email test@example.com --password admin

## To start webserver => airflow webserver -p 8080

dag_script.py <== scheduler script
|
from airflow import DAG
from airflow.operators.python import PythonOperator

with DAG() as obj:
    dag_id=<userdefined_ dagName>
    start_date=datetime(start up date) YYYY,MM,DD
    schedule_interval='@daily' 5minutes '@weekly' 
    catchup=<bool>
             |->True => 2025,11,05 - schedule date
                            |
                       start project on 17th Nov
    PythonOperator(task_id=<>,python_callable=<functionName>)
     <or>
    BashOperator()

----------------------------------------------// ~/airflow/dags/dag_script.py

In [ ]:
(lab2) student@paka:~$ mkdir airflow-activity
(lab2) student@paka:~$ cd airflow-activity/
(lab2) student@paka:~/airflow-activity$
(lab2) student@paka:~/airflow-activity$ vi etl_dag.py
(lab2) student@paka:~/airflow-activity$ vi mydb.py
(lab2) student@paka:~/airflow-activity$ vi pETL.py
(lab2) student@paka:~/airflow-activity$ python mydb.py
(lab2) student@paka:~/airflow-activity$ ls -t
test1.db  pETL.py  mydb.py  etl_dag.py
(lab2) student@paka:~/airflow-activity$ python pETL.py
Data written to /home/student/extract/etl_output_20251105124821.csv
(lab2) student@paka:~/airflow-activity$ pwd
/home/student/airflow-activity
(lab2) student@paka:~/airflow-activity$ vi wrapper_script.sh
(lab2) student@paka:~/airflow-activity$ cat wrapper_script.sh
python /home/student/airflow-activity pETL.py
(lab2) student@paka:~/airflow-activity$ chmod a+x wrapper_script.sh
(lab2) student@paka:~/airflow-activity$ ls
etl_dag.py  mydb.py  pETL.py  test1.db  wrapper_script.sh
(lab2) student@paka:~/airflow-activity$ cp etl_dag.py ~/airflow/dags/
(lab2) student@paka:~/airflow-activity$


In [ ]:
################## end of the day3 ############################